In [1]:
!pip install -qqq -U transformers datasets huggingface_hub accelerate bitsandbytes tqdm --progress-bar off
!FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install -qqq -U flash-attn --no-build-isolation pip install flash-attn --progress-bar off

In [2]:
from huggingface_hub import login
login()

In [3]:
import transformers
import torch
from peft import PeftModel

# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

base_model = "meta-llama/Meta-Llama-3-8B"
base2_model = "AlienKevin/Meta-Llama-3-8B-tagllm-lang-1-fixed-embed"
new_model = "AlienKevin/Meta-Llama-3-8B-tagllm-pos-1-fixed-embed"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

num_token_per_tag = 1
lang_tokens = ['<|TOK0|>', '<|TOK1|>', '<|TOK2|>']
pos_tokens = ['<|TOK200|>']

tokenizer = transformers.AutoTokenizer.from_pretrained(base_model, additional_special_tokens=lang_tokens + pos_tokens)

model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)
model.resize_token_embeddings(len(tokenizer) - 1)
model = PeftModel.from_pretrained(model, base2_model)
model = model.merge_and_unload()

model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [4]:
from datasets import load_dataset, features

def patch_v(tag):
    if tag == 'V':
        return 'VERB'
    else:
        return tag

def get_dataset(num_existing_tokens=0, num_token_per_tag=1):
    dataset = load_dataset("hkcancor", "default")

    single_lang = ["eng", "yue", "cmn"]

    tag_name_dict = {}
    for lang in single_lang:
        tag_name_dict[lang] = "".join([f'<|TOK{i}|>' for i in range(num_existing_tokens, num_existing_tokens + num_token_per_tag)])
        num_existing_tokens += num_token_per_tag

    source_upos = dataset['train'].features["pos_tags_ud"].feature
    print("Source upos:", source_upos)

    def preprocess_function(example):
        example["input"] = tag_name_dict["yue"] + ' '.join(example["tokens"]) + "\n" + \
            ''.join(pos_tokens) + ' '.join(patch_v(source_upos.int2str(tag)).lower()
            for tag in example["pos_tags_ud"])
        return example
    
    dataset['train'] = dataset['train'].map(preprocess_function, remove_columns=
        ['tokens', 'conversation_id', 'pos_tags_prf', 'pos_tags_ud', 'speaker', 'transcriptions', 'turn_number', 'tokens'])

    num_examples = 10
    
    dataset['train'] = dataset['train'].shuffle(42)
    dataset['train'] = dataset['train'].select(range(num_examples))
    prompt = '\n'.join(example['input'] for example in dataset['train'].take(num_examples)) + '\n'
    print(prompt)
    
    dataset = load_dataset("universal_dependencies", "yue_hk")
    test_dataset = dataset["test"]

    def preprocess_function(examples):
        examples["inputs"] = [prompt + tag_name_dict["yue"] + ' '.join(examples["tokens"][i]) + "\n" + ''.join(pos_tokens) for i in range(len(examples["tokens"]))]
        return examples
    
    test_dataset = test_dataset.map(preprocess_function, remove_columns=
        ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'], batched=True)

    return prompt, test_dataset, tag_name_dict

In [5]:
prompt, eval_dataset, tag_name_dict = get_dataset(num_token_per_tag=num_token_per_tag)

Source upos: ClassLabel(names=['NUM', 'ADP', 'INTJ', 'PROPN', 'ADJ', 'V', 'DET', 'ADV', 'CCONJ', 'PRON', 'X', 'PART', 'AUX', 'VERB', 'NOUN', 'PUNCT'], id=None)
<|TOK1|>啲 - 啲 sales 嘅 質素 下降 ， 會 唔 會 好似 同 醫管局 嗰啲 …
<|TOK200|>noun punct noun noun part noun verb punct aux adv aux verb adp propn pron punct
<|TOK1|>唉 ， 有 乜嘢 辦法 啊 ， 興 吖 嗎 ， 咁 興 啊 。
<|TOK200|>intj punct verb pron noun part punct verb part part punct adv verb part punct
<|TOK1|>夠 嚹 。
<|TOK200|>verb part punct
<|TOK1|>噉 另外 就 同 佢 太太 之間 呢 亦都 有 個 中年 婚姻 危機 𡃉 。 即係 覺得 太太 ， 冇 理由 我 太太 會 重 - 重 愛 我 吖 。 即係 我 事業 又 唔 得 ， 要 樣 冇 樣 ， 要 錢 冇 錢 噉樣 。 噉 另外 有 個 黑人 嘅 。 就 係 鬍鬚 阿伯 ， 就 成 五十 歲 嘅 。 點解 會 揾 到 佢 ？ 因為 佢 要 揾 個 人 教 跳舞 吖 嗎 。 而 哩個 阿伯 係 識 跳 所有 爵士舞 ， 只不過 係 因為 骨頭 硬 。 即係 跳 起 身 ， 跌落 地下 之後 就 起 唔 到 身 𡃉 嘞 。 但係 之前 嗰 段 係 得 嘅 。
<|TOK200|>cconj cconj adv adp pron noun adv part adv verb noun noun noun noun part punct cconj verb noun punct verb noun pron noun aux adv punct adv verb pron part punct cconj pron noun adv adv verb punct aux noun verb noun punct aux noun

In [6]:
eval_dataset

Dataset({
    features: ['inputs'],
    num_rows: 1004
})

In [7]:
eval_dataset[:2]

{'inputs': ['<|TOK1|>啲 - 啲 sales 嘅 質素 下降 ， 會 唔 會 好似 同 醫管局 嗰啲 …\n<|TOK200|>noun punct noun noun part noun verb punct aux adv aux verb adp propn pron punct\n<|TOK1|>唉 ， 有 乜嘢 辦法 啊 ， 興 吖 嗎 ， 咁 興 啊 。\n<|TOK200|>intj punct verb pron noun part punct verb part part punct adv verb part punct\n<|TOK1|>夠 嚹 。\n<|TOK200|>verb part punct\n<|TOK1|>噉 另外 就 同 佢 太太 之間 呢 亦都 有 個 中年 婚姻 危機 𡃉 。 即係 覺得 太太 ， 冇 理由 我 太太 會 重 - 重 愛 我 吖 。 即係 我 事業 又 唔 得 ， 要 樣 冇 樣 ， 要 錢 冇 錢 噉樣 。 噉 另外 有 個 黑人 嘅 。 就 係 鬍鬚 阿伯 ， 就 成 五十 歲 嘅 。 點解 會 揾 到 佢 ？ 因為 佢 要 揾 個 人 教 跳舞 吖 嗎 。 而 哩個 阿伯 係 識 跳 所有 爵士舞 ， 只不過 係 因為 骨頭 硬 。 即係 跳 起 身 ， 跌落 地下 之後 就 起 唔 到 身 𡃉 嘞 。 但係 之前 嗰 段 係 得 嘅 。\n<|TOK200|>cconj cconj adv adp pron noun adv part adv verb noun noun noun noun part punct cconj verb noun punct verb noun pron noun aux adv punct adv verb pron part punct cconj pron noun adv adv verb punct aux noun verb noun punct aux noun verb noun cconj punct cconj cconj verb noun noun part punct adv verb noun noun punct adv num num noun part punct pron aux verb part pron pu

In [8]:
# https://huggingface.co/PygmalionAI/pygmalion-6b/discussions/25#64387bf26c8841ba74e7d9c0
from transformers import StoppingCriteria

class TranslationStoppingCriteria(StoppingCriteria):
    def __init__(self, prompt):
        self.prompt = prompt
        
    def __call__(self, input_ids, scores, **kwargs):
        # Get the generated text as a string
        generated_text = tokenizer.decode(input_ids[0])
        generated_text = generated_text.removeprefix(prompt)
        if generated_text.endswith('\n'):
            return True  # Stop generation
        return False  # Continue generation
    
    def __len__(self):
        return 1
    
    def __iter__(self):
        yield self

In [ ]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm
import json

outputs = pipeline(
    KeyDataset(eval_dataset, "inputs"),
    max_new_tokens=128,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    stopping_criteria=TranslationStoppingCriteria(prompt),
    pad_token_id=tokenizer.eos_token_id,
)

def parse_translation(text):
    lines = text.strip().split('\n')
    result = { 'langs': [], 'sents': [] }

    assert(len(pos_tokens) == 1)
    # tag_dict = { 'yue': tag_name_dict['yue'], 'pos': pos_tokens[0] }
    tag_dict = { 'yue': 'yue:', 'pos': 'pos:' }
    
    for line in lines:
        if len(line.strip()) > 0:
            for tag in tag_dict.values():
                if line.startswith(tag):
                    lang = next(lang for lang, t in tag_dict.items() if t == tag)
                    content = line.removeprefix(tag)
                    result['langs'].append(lang)
                    result['sents'].append(content.strip())
                    break
    
    return result

with open(f'pos_{new_model.split("/")[1]}.jsonl', 'w+') as f:
    for output in tqdm(outputs, total=len(eval_dataset)):
        generated_text = output[0]['generated_text']
        generated_text = generated_text.removeprefix(prompt)
        print(generated_text)
        f.write(json.dumps(parse_translation(generated_text)) + '\n')
        f.flush()

  0%|          | 1/1004 [00:07<2:03:03,  7.36s/it]

<|TOK1|>你 喺度 搵 乜嘢 呀 ？
<|TOK200|>cconj pron adv noun part punct cconj pron noun part num noun part punct coun noun part cconj pron noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun part coun noun


  0%|          | 2/1004 [00:14<1:59:37,  7.16s/it]

<|TOK1|>咪 執 返 啲 嘢 去 阿哥 個 新 屋 度 囖 。
<|TOK200|>cconj pron verb part num noun part punct cconj pron verb part num noun verb part num noun part punct counj part verb num noun part punct counj part verb part num noun part num noun part punct counj part num noun part num noun part punct counj part num noun part num noun part num noun part num noun part num noun part num noun part num noun part num noun part num num noun part num noun part num noun part num num noun part num noun part num num noun part num num noun part num num noun part num num noun part num num noun part num noun part num num noun part num


  0%|          | 3/1004 [00:21<1:58:34,  7.11s/it]

...

<|TOK1|>姐姐 呀 ， 啲 錢 畀 我 得 㗎 嘞 ！
<|TOK200|>counj punct cconj pron verb part num noun part punct cconj pron noun part num noun part num noun part cjon pron noun part cunj punct cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cunj cjj part cun


  5%|▌         | 52/1004 [05:53<1:50:52,  6.99s/it]

<|TOK1|>爺爺 呀 ， 呢啲 畀 你 嘅 ！
<|TOK200|>cconj pron adv part num noun part punct cconj pron verb part num noun part cjon pron verb part num noun part num noun part cjon pron verb part num noun part cjon pron verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb part num noun part cjon pro verb


  5%|▌         | 53/1004 [06:00<1:50:54,  7.00s/it]

<|TOK1|>嗰啲 呢 ？
<|TOK200|>coun noun part verb punct cconj pron noun part verb part punct cconj pron verb noun part punct cunjousnoun part num noun part cunjousnoun partnum noun part cunjousnoun part num noun part cunjousnoun part cunjous noun part cunjous noun part cunjous noun part cunjous part cunjous noun part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunjous part cunj
